# Introduction to basic functionalities of HOOMD-Organics

## Overview:

This tutorial introduces some basic functionalities of the HOOMD-Organics package including:

- Initializing molecule structures
- Assembling molecules in a box
- Applying force fields
- Running MD simulations using different methods




## Step-by-step guide for running MD simulation for a box of  Polyphenylene sulfide (PPS) polymers:
In this tutorial, we will run a molecular dynamics simulation of Polyphenylene sulfide (PPS) polymers using the HOOMD-Organics package.
HOOMD-Organics uses [`HOOMD-blue`](https://hoomd-blue.readthedocs.io/en/v4.1.0/) simulation engine in the background to run different methods of simulation and utilizes many functionalities from [`mBuild`](https://mbuild.mosdef.org/en/stable/) and [GMSO](https://gmso.mosdef.org/en/stable/) packages to initialize the structures, apply force-fields and prepare necessary information to run a simulation.

In summary, the `HOOMD-Organics` package has three main classes:

-  `Molecule`: This class is used to define the structure of a molecule (for example the structure of a polymer built from a monomer). This class allows us to implement recipies for building complex structures.

- `System`: This class is used to assemble molecules into a box and to capture the initial `gsd` snapshot of the system. It also applies the forcefiled to the system and prepares the required forces for the simulation.

- `Simulation`: This class is used to run the simulation using the `HOOMD-blue` simulation engine. In order to initialize a simulation, a `gsd` snapshot of the system and a list of `Hoomd` forces are required.

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import hoomd_organics

from hoomd_organics import Pack, Simulation

### Step 1: Initializing the Molecule
In this example, we are using the pre-defined recipe for building PPS molecules defined in `HOOMD_Organics` library. The `PPS` class is a subclass of the `Molecule` class. This class includes all the necessary information for building the PPS chains, including the monomer structure and how the monomers bond to create a chain. All we need to specify is the polymer chain length and how many polymer chains we want to create. In this example, we will create a system of 3 PPS chains with a length of 8.


In [40]:
from hoomd_organics.library import PPS

molecule = PPS(num_mols=3, lengths=8)

### Step 2: Initializing the System
In this step, we will use the `Pack` class, which is a subclass of the `System` class to pack a box of PPS molecules in a random fashion given a density. The system class creates the box, organizes molecules within the box and applies the forcefield (if provided) to the system and creates
the initial state of the system in form a HOOMD snapshot. 
When a forcefield object (obtained from XML-based forcefields) is provided for the `System` class, this class automatically generates the list of force objects defining bonded and non-bonded interactions by applying the forcefield to the system. 

Alternatively, users can initiate their own custom list of HOOMD force objects. In such cases, they should skip passing the `force_field` parameter during system setup and instead provide it directly to the `Simulation` class in the subsequent step. This approach allows for a greater flexibility in customizing the force interactions within the simulation, specially in cases where XML-based forcefield are not available. For more examples, see coarse graining tutorial. (link to tutorial)



In this example, the `Pack` class invokes mBuild's `fill_box` method in the background, which efficiently places molecules within a box in a randomized manner without overlaps. We use the pre-defined `OPLS` force-field class which was created from the [OPLS](https://en.wikipedia.org/wiki/OPLS) XML forcefield to parameterize particle interactions.

The `HOOMD-Organics` library offers some commonly used forcefields that can be employed to parameterize the interactions within specific systems. Please refer to `hoomd_organics.library.forcefields` for more examples.



We also specify the `r_cut` parameter, which is the cutoff distance for the non-bonded interactions. If `auto_scale` is set to `True`, all the parameters defined in forces will be scaled. For example, all the `epsilon` values of Leonard-Jones potentials are scaled based on the maximum `epsilon` value. 

In [53]:
from hoomd_organics.library import OPLS_AA_PPS

system = Pack(molecules=molecule, force_field=OPLS_AA_PPS(), density=0.1, r_cut=2.5, auto_scale=True)

The initial snapshot can be acquired from the `system.hoomd_snapshot` object.


In [42]:
system.hoomd_snapshot

The list of HOOMD force objects applied to the system can also be obtained by accessing the `system.hoomd_forcefield` attribute. These forces correspond to the bonded and non-bonded interactions parameterized from the `OPLS` forcefield.

In [43]:
system.hoomd_forcefield

Now, let's examine the parameters of the LJ pair force. As you can see, the values of `epsilon` have been scaled to fall within the range of  0 to 1. The scaling factor for the `epsilon` parameter, which is expressed in units of energy, can be retrieved  from `system.reference_energy`. We will discuss reference values in details in the next tutorials. 

In [44]:
lj_force = system.hoomd_forcefield[3]

dict(lj_force.params)

{('ca',
  'ca'): _HOOMDDict{'epsilon': 0.16470588235294117, 'sigma': 0.9861111111111112},
 ('ca',
  's'): _HOOMDDict{'epsilon': 0.3709148887161047, 'sigma': 0.9930312739844155},
 ('ca',
  'ha'): _HOOMDDict{'epsilon': 0.10782531046954917, 'sigma': 0.8141779918845362},
 ('ca',
  'sh'): _HOOMDDict{'epsilon': 0.4058397249567139, 'sigma': 0.9930312739844155},
 ('ca', 'hs'): _HOOMDDict{'epsilon': 0.0, 'sigma': 0.0},
 ('s', 's'): _HOOMDDict{'epsilon': 0.835294117647059, 'sigma': 1.0},
 ('ha',
  's'): _HOOMDDict{'epsilon': 0.24282079341823876, 'sigma': 0.8198915917499229},
 ('s', 'sh'): _HOOMDDict{'epsilon': 0.9139442639718568, 'sigma': 1.0},
 ('hs', 's'): _HOOMDDict{'epsilon': 0.0, 'sigma': 0.0},
 ('ha',
  'ha'): _HOOMDDict{'epsilon': 0.07058823529411765, 'sigma': 0.6722222222222223},
 ('ha',
  'sh'): _HOOMDDict{'epsilon': 0.2656844656620286, 'sigma': 0.8198915917499229},
 ('ha', 'hs'): _HOOMDDict{'epsilon': 0.0, 'sigma': 0.0},
 ('sh', 'sh'): _HOOMDDict{'epsilon': 1.0, 'sigma': 1.0},
 ('hs', 

In [45]:
system.reference_energy

unyt_quantity(1.7782, 'kJ/mol')

### Step 3: Running the Simulation

Using the snapshot and force objects provided by the `System` class in the previous step, we can proceed to initialize the simulation. The `Simulation` class, a subclass of `hoomd.Simulation`, offers additiona;l features and functionalities that automates simulation methods such as updating box volume, welding process and tensile tests. 



Additionally, this class logs snapshots of the simulation in form of a `gsd` trajectory file while running simulation. The `gsd_write_freq` specifies the frequency of saving snapshots into the gsd file. This class also logs other simulation data such as timestep, potential energy, kinetic temperature, pressure and volume into a text file. The frequency for logging these information can be set by `log_write_freq` parameter.

In [58]:
sim = Simulation(initial_state=system.hoomd_snapshot, forcefield=system.hoomd_forcefield, gsd_write_freq=100, log_write_freq=100)

Initializing simulation state from a snapshot.


Let's run the simulation for 1000 time steps using the NVT ensemble at a given scaled temperature of 1.0.

In [47]:
sim.run_NVT(n_steps=1000, kT=1.0, tau_kt=0.01)

*Warning*: charge.pppm: system is not neutral and unscreened interactions are calculated, the net charge is -214.352
*Warning*: charge.pppm: RMS error of 0.398055 is probably too high! 0.397977 0.398055


Step 100 of 1000; TPS: 347.2; ETA: 0.0 minutes
Step 200 of 1000; TPS: 573.99; ETA: 0.0 minutes
Step 300 of 1000; TPS: 733.64; ETA: 0.0 minutes
Step 400 of 1000; TPS: 852.98; ETA: 0.0 minutes
Step 500 of 1000; TPS: 944.92; ETA: 0.0 minutes
Step 600 of 1000; TPS: 1017.91; ETA: 0.0 minutes
Step 700 of 1000; TPS: 1076.69; ETA: 0.0 minutes
Step 800 of 1000; TPS: 1126.6; ETA: 0.0 minutes
Step 900 of 1000; TPS: 1168.76; ETA: 0.0 minutes


The simulation class also allows user to run the simulation under different conditions/ensembles such as NPT ensemble, NVE ensemble, Langevin dynamics. Checkout `hoomd_organics/base/simulation.py` for more functionalities.

We can access simulation box length and shrink the volume to reach the desired density specified in the system. The `System` class calculates the `target_box` lengths based on the system mass and specified density.

In [55]:
sim.box_lengths

array([48.73535538, 48.73535538, 48.73535538])

In [56]:
system.target_box

array([3.50894566, 3.50894566, 3.50894566])

In [59]:
sim.run_update_volume(n_steps=10000, period=1, kT=1, tau_kt=1, final_box_lengths=system.target_box)

*Warning*: charge.pppm: system is not neutral and unscreened interactions are calculated, the net charge is -214.352
*Warning*: charge.pppm: RMS error of 0.295878 is probably too high! 0.295878 0.29587


Step 100 of 10000; TPS: 381.23; ETA: 0.4 minutes
Step 200 of 10000; TPS: 605.37; ETA: 0.3 minutes
Step 300 of 10000; TPS: 755.52; ETA: 0.2 minutes
Step 400 of 10000; TPS: 859.56; ETA: 0.2 minutes
Step 500 of 10000; TPS: 937.56; ETA: 0.2 minutes
Step 600 of 10000; TPS: 994.2; ETA: 0.2 minutes
Step 700 of 10000; TPS: 1045.78; ETA: 0.1 minutes
Step 800 of 10000; TPS: 1086.93; ETA: 0.1 minutes
Step 900 of 10000; TPS: 1121.21; ETA: 0.1 minutes
Step 1000 of 10000; TPS: 1151.44; ETA: 0.1 minutes
Step 1100 of 10000; TPS: 1173.12; ETA: 0.1 minutes
Step 1200 of 10000; TPS: 1192.6; ETA: 0.1 minutes
Step 1300 of 10000; TPS: 1192.42; ETA: 0.1 minutes
Step 1400 of 10000; TPS: 1188.54; ETA: 0.1 minutes
Step 1500 of 10000; TPS: 1201.85; ETA: 0.1 minutes
Step 1600 of 10000; TPS: 1215.09; ETA: 0.1 minutes
Step 1700 of 10000; TPS: 1226.38; ETA: 0.1 minutes
Step 1800 of 10000; TPS: 1235.08; ETA: 0.1 minutes
Step 1900 of 10000; TPS: 1241.35; ETA: 0.1 minutes
Step 2000 of 10000; TPS: 1249.25; ETA: 0.1 minut

RuntimeError: nlist: Simulation box is too small! Particles would be interacting with themselves.rmax=2.9
nearest_plane_distance.x=5.65268
nearest_plane_distance.y=5.65268
nearest_plane_distance.z=5.65268



In the upcoming tutorials, we will explore a selection of features offered by the `HOOMD-Organics` package, highlighting how they can be customized to meet specific research requirements.